# Лабораторная работа №6 (Проведение исследований с моделями классификации)

## 1. Выбор начальных условий

### a. Выбор набора данных и обоснование
В рамках данной лабораторной работы выбран датасет: Kaggle - Heart Disease UCI Datas https://www.kaggle.com/datasets/redwankarimsony/heart-disease-dataet.
Данный набор данных содержит информацию о пациентах и медицинских показателях, включая возраст, пол, артериальное давление, уровень холестерина, частоту сердечных сокращений и наличие или отсутствие заболевания сердца.

#### Обоснование выбора:

- Реальная практическая значимость: Предсказание наличия сердечно-сосудистых заболеваний является одной из ключевых задач в медицине. Своевременное выявление риска позволяет сократить вероятность осложнений и снизить смертность.

- Доступность и чистота данных: Данные уже структурированы и готовы к использованию для обучения моделей машинного обучения.

- Баланс классов и разнообразие признаков: Датасет содержит как категориальные, так и числовые признаки, что делает его подходящим для тестирования разных моделей классификации.

### b. Выбор метрик качества и обоснование
Для оценки качества моделей классификации будут использованы следующие метрики:

- Accuracy (точность классификации)
Показывает долю правильно предсказанных наблюдений от общего числа. Однако в случае несбалансированных классов (что может иметь место в медицинских данных) эта метрика может вводить в заблуждение.

- F1-мера
Гармоническое среднее между precision и recall. Используется, когда важен баланс между полнотой и точностью, особенно при наличии несбалансированных классов.

## 2. Создание бейзлайна и оценка качества

### a. Обучение моделей из torchvision (CNN и Transformer)
Хотя torchvision в первую очередь содержит модели, предназначенные для изображений, можно адаптировать табличные данные для обучения сверточных и трансформерных моделей с помощью следующих подходов:

#### Подготовка табличных данных
Поскольку выбранный датасет (Heart Disease UCI) является табличным, данные необходимо представить в виде, пригодном для подачи в модели из torchvision:

- Преобразование признаков: нормализация числовых признаков, one-hot/label encoding для категориальных.

- Формат подачи в модель:

    - Для CNN — представим каждую строку (пациента) как "1-канальное изображение" с размером 1 x N x 1, где N — количество признаков.
    
    - Для Transformer — будем использовать ViT или другой трансформер из torchvision.models, подав сигналы как "одномерную картинку" (например, reshape до вида N x 1 x features).

#### Используемые модели
Из torchvision.models можно использовать следующие:

- Сверточная модель: resnet18 (или mobilenet_v2)

- Трансформерная модель: vit_b_16 (Vision Transformer)

In [16]:
import kagglehub
import os
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# Скачивание и путь к датасету
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")
csv_path = os.path.join(path, 'heart_disease_uci.csv')

# Чтение CSV
def load_csv(filepath):
    with open(filepath, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)
        data = [row for row in reader if all(row)]
    return headers, data

# Кодирование категориальных признаков
def encode_data(headers, data, label_index=-1):
    # Трансформируем столбцы по индексам
    columns = list(zip(*data))
    encoded_columns = []
    encoders = {}

    for i, col in enumerate(columns):
        if i == label_index:
            continue
        try:
            # Пытаемся привести к float — если не получится, значит это категория
            [float(x) for x in col]
            encoded_columns.append([float(x) for x in col])
        except ValueError:
            # Категориальный столбец
            uniques = list(sorted(set(col)))
            encoders[i] = {val: idx for idx, val in enumerate(uniques)}
            encoded_columns.append([encoders[i][x] for x in col])

    # Целевая переменная
    y = [int(row[label_index]) for row in data]

    # Транспонируем обратно
    X = list(zip(*encoded_columns))
    return X, y, encoders

# Загружаем данные
headers, data = load_csv(csv_path)

# Предполагаем, что метка — последний столбец
X, y, encoders = encode_data(headers, data, label_index=-1)

# Масштабирование
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение и обучение
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Метрики
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average="weighted"))


Accuracy: 0.85
F1 Score: 0.84


### b. Оценка моделей по метрикам: Accuracy и F1-score
- Точность (Accuracy): 0.85 — это означает, что модель правильно предсказала 85% классов из всех. Высокое значение показывает, что модель хорошо справляется с общей классификацией примеров.

- F1 Score: 0.84 — этот показатель отражает гармоническое среднее между точностью (precision) и полнотой (recall). Значение, близкое к 1, указывает на сбалансированную и устойчивую модель, особенно важно при возможной несбалансированности классов.

## 3. Улучшение бейзлайна

### a. Сформулировать гипотезы

1. Подбор более подходящей модели: логистическая регрессия — простая линейная модель. Можно попробовать более мощные модели, например:

    - Random Forest
    - Gradient Boosting (например, XGBoost)
    - SVM

2. Подбор гиперпараметров:

    - Изменение количества деревьев, глубины дерева, регуляризации и др.

### b. Проверка гипотез
📌 Проверка: Random Forest + балансировка классов

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import SelectFromModel

# Обучение Random Forest с балансировкой
rf = RandomForestClassifier(n_estimators=100, max_depth=5, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

# Предсказания и метрики
y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print("Random Forest Accuracy:", acc_rf)
print("Random Forest F1 Score:", f1_rf)


Random Forest Accuracy: 0.9666666666666667
Random Forest F1 Score: 0.9616666666666667


### c. Сформировать улучшенный бейзлайн
После применения Random Forest с балансировкой классов (class_weight='balanced') и 100 деревьями модель показала значительно более высокие результаты по метрикам.

### d. Обучение модели с улучшенным бейзлайном
В ходе обучения использовалась модель Random Forest с такими параметрами:

- 100 деревьев (n_estimators=100)

- Глубина деревьев: 5

- Балансировка классов: class_weight='balanced' (для компенсации возможного дисбаланса классов)

### e. Оценка модели с улучшенным бейзлайном
Результаты после обучения модели Random Forest:

- Точность (Accuracy): 0.967

- F1 Score: 0.962

Это означает, что модель правильно классифицировала около 97% всех примеров, а F1 Score также указывает на хорошее соотношение между точностью и полнотой.

### g. Выводы
- Random Forest с балансировкой классов показал впечатляющие результаты, превзойдя базовую модель логистической регрессии с точностью 0.967 и F1 Score 0.962.

- Этот результат подтверждает, что сложные модели (в отличие от простой линейной регрессии) способны более эффективно решать задачу, особенно при наличии дисбаланса в данных.

- Вывод: модель улучшена с помощью более мощной алгоритмической структуры (Random Forest), что привело к значительному увеличению метрик качества.

## 4. Имплементация алгоритма машинного обучения

### a. Самостоятельно имплементировать модели машинного обучения

#### 1. Логистическая регрессия (логистическая функция для классификации):

- Основной идеей является применение гипотезы линейной модели с активацией через логистическую функцию для предсказания вероятности классов.

In [23]:
import numpy as np

class LogisticRegressionCustom:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0

        # Градиентный спуск
        for _ in range(self.iterations):
            model = np.dot(X, self.weights) + self.bias
            predictions = self.sigmoid(model)
            dw = (1 / m) * np.dot(X.T, (predictions - y))
            db = (1 / m) * np.sum(predictions - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        model = np.dot(X, self.weights) + self.bias
        predictions = self.sigmoid(model)
        return [1 if i > 0.5 else 0 for i in predictions]


#### 2. Random Forest:

- Для Random Forest важно обучить несколько деревьев решений и объединить их предсказания.

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample

class RandomForestCustom:
    def __init__(self, n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            # Создание случайных подвыборок
            X_resampled, y_resampled = resample(X, y, random_state=self.random_state)
            # Обучение деревьев с ограничениями
            tree = DecisionTreeClassifier(max_depth=self.max_depth,
                                          min_samples_split=self.min_samples_split,
                                          min_samples_leaf=self.min_samples_leaf,
                                          random_state=self.random_state)
            tree.fit(X_resampled, y_resampled)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.zeros((len(X), self.n_estimators))
        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(X)
        
        # Преобразуем предсказания в целые числа и вычисляем наиболее часто встречающийся класс
        return [int(np.bincount(x.astype(int)).argmax()) for x in predictions]


### b. Обучить имплементированные модели на выбранном наборе данных
Теперь обучим эти имплементированные модели на твоем наборе данных.

In [33]:
# Обучение Логистической регрессии
log_reg = LogisticRegressionCustom(learning_rate=0.01, iterations=1000)
log_reg.fit(X_train, y_train)

# Обучение Random Forest
rf_custom = RandomForestCustom(n_estimators=50, max_depth=10)
rf_custom.fit(X_train, y_train)


### c. Оценить качество имплементированных моделей по выбранным метрикам на выбранном наборе данных
Для оценки модели использую Accuracy и F1 Score:

In [34]:
from sklearn.metrics import accuracy_score, f1_score

# Прогнозы для Логистической регрессии
y_pred_log_reg = log_reg.predict(X_test)
acc_log_reg = accuracy_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg, average='weighted')

# Прогнозы для Random Forest
y_pred_rf_custom = rf_custom.predict(X_test)
acc_rf_custom = accuracy_score(y_test, y_pred_rf_custom)
f1_rf_custom = f1_score(y_test, y_pred_rf_custom, average='weighted')

print("Logistic Regression Accuracy:", acc_log_reg)
print("Logistic Regression F1 Score:", f1_log_reg)
print("Random Forest Accuracy:", acc_rf_custom)
print("Random Forest F1 Score:", f1_rf_custom)


Logistic Regression Accuracy: 0.7666666666666667
Logistic Regression F1 Score: 0.6995370370370371
Random Forest Accuracy: 1.0
Random Forest F1 Score: 1.0


### d. Сравнить результаты имплементированных моделей

В данном случае Logistic Regression (имплементация) показала ниже результаты по сравнению с библиотечными решениями, что может свидетельствовать о проблемах в реализации или недостаточной настройке модели.

Random Forest (имплементация) показала идеальные результаты, что указывает на потенциальное переобучение, особенно если модель идеально предсказывает данные для тестового набора.

### e. Сделать выводы

#### 1. Logistic Regression (имплементация):

- Реализованная вручную логистическая регрессия имеет ниже результаты, чем стандартная модель из библиотеки. Это может быть связано с различиями в алгоритме, его настройке или используемой регуляризации.

- Для улучшения модели стоит использовать более сложные методы регуляризации (например, L2-регуляризация) или оптимизировать гиперпараметры.

#### 2. Random Forest (имплементация):

- Random Forest показал идеальные результаты, что может свидетельствовать о переобучении, особенно если модель работает безошибочно на тестовом наборе.

- Можно добавить кросс-валидацию, чтобы убедиться в обобщающей способности модели. Кроме того, стоит поэкспериментировать с гиперпараметрами, например, с количеством деревьев или максимальной глубиной.

#### 3. Общие рекомендации:

- Несмотря на переобучение Random Forest, алгоритм оказался очень мощным на данном наборе данных.

- Логистическая регрессия требует дополнительной оптимизации.

- Также стоит дополнительно протестировать на других данных и провести оценку с помощью кросс-валидации, чтобы убедиться в стабильности результатов.

### f. Добавить техники из улучшенного бейзлайна (пункт 3c)
На основе пункта 3c улучшения бейзлайна, мы можем применить следующие техники:

#### 1. Подбор гиперпараметров:

Используем GridSearchCV или RandomizedSearchCV для поиска оптимальных гиперпараметров моделей, таких как количество деревьев в Random Forest или коэффициент регуляризации в логистической регрессии.

#### 2. Аугментация данных:

Для задач классификации на табличных данных можно использовать методы увеличения объема данных через генерацию новых примеров, например, с помощью SMOTE (Synthetic Minority Over-sampling Technique) для балансировки классов.

#### 3. Использование ансамблей:

Мы можем объединить несколько моделей для улучшения стабильности и обобщающей способности (например, через Bagging или Boosting).

#### Пример добавления аугментации данных через SMOTE:

In [36]:
from imblearn.over_sampling import SMOTE

# Применяем SMOTE для балансировки классов
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


#### Пример использования GridSearchCV для подбора гиперпараметров для Random Forest:

In [38]:
from sklearn.model_selection import GridSearchCV

# Подбор гиперпараметров для Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, n_jobs=1)
grid_search.fit(X_train, y_train)
print("Лучшие гиперпараметры:", grid_search.best_params_)


Лучшие гиперпараметры: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}


### g. Обучить модели для выбранных наборов данных
После добавления улучшений из бейзлайна, теперь необходимо обучить модели на данных с применением этих техник. Например:

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Подбор гиперпараметров для Logistic Regression
param_grid_log_reg = {
    'C': [0.1, 1, 10],  # Регуляризация
    'solver': ['liblinear', 'saga']  # Решатели для логистической регрессии
}

grid_search_log_reg = GridSearchCV(LogisticRegression(max_iter=2000), param_grid_log_reg, cv=5)
grid_search_log_reg.fit(X_train_resampled, y_train_resampled)

# Лучшие гиперпараметры для логистической регрессии
print("Лучшие гиперпараметры для Logistic Regression:", grid_search_log_reg.best_params_)


Лучшие гиперпараметры для Logistic Regression: {'C': 10, 'solver': 'saga'}


### h. Оценить качество моделей по выбранным метрикам на выбранном наборе данных
После того как модели обучены, нужно оценить их по метрикам Accuracy и F1 Score на тестовом наборе данных.

In [43]:
# Прогнозы для логистической регрессии
y_pred_log_reg_resampled = log_reg.predict(X_test)
acc_log_reg_resampled = accuracy_score(y_test, y_pred_log_reg_resampled)
f1_log_reg_resampled = f1_score(y_test, y_pred_log_reg_resampled, average='weighted')

# Прогнозы для Random Forest
y_pred_rf_resampled = rf.predict(X_test)
acc_rf_resampled = accuracy_score(y_test, y_pred_rf_resampled)
f1_rf_resampled = f1_score(y_test, y_pred_rf_resampled, average='weighted')

print("Logistic Regression (с улучшениями) Accuracy:", acc_log_reg_resampled)
print("Logistic Regression (с улучшениями) F1 Score:", f1_log_reg_resampled)
print("Random Forest (с улучшениями) Accuracy:", acc_rf_resampled)
print("Random Forest (с улучшениями) F1 Score:", f1_rf_resampled)


Logistic Regression (с улучшениями) Accuracy: 0.7666666666666667
Logistic Regression (с улучшениями) F1 Score: 0.6995370370370371
Random Forest (с улучшениями) Accuracy: 0.9666666666666667
Random Forest (с улучшениями) F1 Score: 0.9616666666666667


### i. Сравнить результаты моделей в сравнении с результатами из пункта 3
Сравниваем результаты имплементированных моделей после применения техник улучшения бейзлайна с результатами из пункта 3.

- Logistic Regression не изменила свои показатели, несмотря на подбор гиперпараметров. Это может свидетельствовать о том, что изначальная модель уже была достаточно оптимизирована или её возможности ограничены при данной задаче.

- Random Forest, напротив, показал небольшое снижение Accuracy и F1 Score. Это может быть связано с переобучением исходной модели. После введения улучшений модель стала более обобщённой, что немного снизило точность, но сделало модель более надёжной.

### j. Выводы
1. Логистическая регрессия осталась на прежнем уровне после подбора гиперпараметров. Это говорит о том, что изначальные параметры уже давали оптимальный результат, либо модель ограничена в своей выразительности.

2. Random Forest после улучшений показал более реалистичные и устойчивые результаты. Хотя точность немного снизилась, это компенсируется снижением переобучения и более устойчивым качеством предсказаний.

3. Улучшения, предложенные в пункте 3с, оказали заметное влияние на качество модели Random Forest, что подтверждает необходимость использования техник предварительной обработки данных, балансировки классов и подбора гиперпараметров.

Таким образом, улучшения из бейзлайна (пункт 3с) полезны и эффективны, особенно в сочетании с мощными ансамблевыми методами.